In [51]:
import numpy as np
import pandas as pd

In [52]:
data = pd.read_csv(r'D:\Data Science\NLP\Fake-New-LSTM-master\train.csv'),

In [53]:
data[0].columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [54]:
data_copy = data[0].copy()

In [55]:
data_copy.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
data_copy.dropna(inplace= True)

In [57]:
data_copy.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [58]:
X = data_copy.drop(['label'], axis = 1)
y = data_copy['label']

In [59]:
X.shape

(18285, 4)

In [60]:
y.shape

(18285,)

In [61]:
X[0:5]['title']

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [62]:
import tensorflow as tf
tf.__version__

'2.4.0'

In [63]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [64]:
##Initialise vocabulary size

voc_size = 5000

### One hot Representation

In [65]:
X.reset_index(inplace = True)

In [66]:
X.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [67]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [70]:
# Data preprocessing.

ps = PorterStemmer()
corpus = []
for i in range(len(X)):
    review = re.sub('[^a-zA-z]', ' ', X['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [71]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [72]:
onehot_rep = [one_hot(word, voc_size) for word in corpus]

onehot_rep

[[1008, 2840, 4477, 4289, 3439, 4245, 1856, 4071, 4805, 4547],
 [4930, 2513, 4033, 2938, 2124, 2595, 733],
 [3473, 3064, 2179, 4874],
 [1321, 671, 1425, 3433, 24, 4194],
 [2959, 2124, 3941, 4758, 859, 4666, 2124, 1088, 1254, 724],
 [1567,
  4805,
  3095,
  3747,
  1266,
  3432,
  3451,
  3461,
  2599,
  801,
  1207,
  1610,
  4703,
  1853,
  733],
 [1121, 4859, 3320, 2977, 1110, 4314, 392, 287, 2291, 4257, 2936],
 [1281, 4682, 4637, 4915, 2065, 599, 3432, 1586, 2291, 4257, 2936],
 [912, 3495, 4808, 3001, 412, 2232, 2931, 3329, 3432, 1199],
 [2905, 2580, 2139, 3824, 286, 1073, 4559, 4778],
 [1524, 2930, 2963, 1916, 585, 379, 1003, 1477, 1560, 3554, 3817],
 [3433, 101, 3439, 2232, 3432, 2065],
 [3615, 2673, 1895, 3963, 1522, 3159, 1404, 3396, 2254],
 [2941, 49, 3409, 3568, 3630, 1591, 4229, 2291, 4257, 2936],
 [4613, 2677, 4705, 693, 28, 2291, 4257, 2936],
 [4561, 4539, 2413, 3281, 3633, 449, 2136, 2647, 1322, 1250],
 [3418, 2943, 2513],
 [2999, 4943, 1284, 2521, 3432, 1872, 1119, 733],


### Word Embedding Representation

In [74]:
sent_len = 20
embedded_docs = pad_sequences(onehot_rep, maxlen = sent_len, padding = 'pre')
embedded_docs

array([[   0,    0,    0, ..., 4071, 4805, 4547],
       [   0,    0,    0, ..., 2124, 2595,  733],
       [   0,    0,    0, ..., 3064, 2179, 4874],
       ...,
       [   0,    0,    0, ..., 2291, 4257, 2936],
       [   0,    0,    0, ..., 1988, 3855, 1430],
       [   0,    0,    0, ..., 3442, 4760,  611]])

In [75]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1008,
       2840, 4477, 4289, 3439, 4245, 1856, 4071, 4805, 4547])

In [79]:
#Creating model
embedding_vector_features = 40

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [80]:
len(embedded_docs), y.shape

(18285, (18285,))

In [81]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

X_final.shape, y_final.shape

((18285, 20), (18285,))

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state = 42)

In [88]:
## Model training

model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 5, batch_size = 5)

Epoch 1/5
2450/2450 [==============================] - 42s 16ms/step - loss: 0.2917 - accuracy: 0.8701 - val_loss: 0.1970 - val_accuracy: 0.9215
Epoch 2/5
2450/2450 [==============================] - 37s 15ms/step - loss: 0.1113 - accuracy: 0.9563 - val_loss: 0.1997 - val_accuracy: 0.9226
Epoch 3/5
2450/2450 [==============================] - 35s 14ms/step - loss: 0.0599 - accuracy: 0.9787 - val_loss: 0.2460 - val_accuracy: 0.9196
Epoch 4/5
2450/2450 [==============================] - 36s 15ms/step - loss: 0.0267 - accuracy: 0.9928 - val_loss: 0.3036 - val_accuracy: 0.9196
Epoch 5/5
2450/2450 [==============================] - 36s 15ms/step - loss: 0.0166 - accuracy: 0.9945 - val_loss: 0.4223 - val_accuracy: 0.9170


In [93]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = model.predict_classes(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Accuracy : {}".format(accuracy_score(y_test, y_pred)))
print("\n Confusion Matrix : \n{}". format(cm))

Accuracy : 0.9169842584921293

 Confusion Matrix : 
[[3140  279]
 [ 222 2394]]
